<a href="https://colab.research.google.com/github/zashari/TextClassification_fromNewsData/blob/main/TextClassification_fromNewsData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# About the Dataset

# Code

In [ ]:
import pandas as pd
import re
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import precision_score, recall_score, roc_auc_score
import numpy as np

## Data Cleaning

In [ ]:
# Load Dataset
data = pd.read_csv("news_data.csv")

# Function to Clean the text and remove stopwords
def clean_text(text):
    # Remove Symbols and Characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Remove words such as "https" dan ".com"
    text = re.sub(r'https|com', '', text)
    # make it lowecase
    text = text.lower()

    # Remove Indonesian Stopworsd
    stopword_factory = StopWordRemoverFactory()
    stopwords = stopword_factory.get_stop_words()
    words = text.split()
    words = [word for word in words if word not in stopwords]

    # join texts that already cleaned from stopdowrd
    text = ' '.join(words)
    return text

# apply the function to the dataset on column 'Text'
data['Text'] = data['Text'].apply(clean_text)

In [ ]:
data

,Judul,Text,Media,Label
0,Pakar Ungkap Peradaban Manusia yang Paling Ber...,peradaban mesir china mesopotamia peradaban te...,CNN Indonesia,Teknologi
1,"Daftar Kuota Internet Medsos Harian Murah, Cek...",telkomsel tri axis tercatat paket internet med...,CNN Indonesia,Teknologi
2,4 Bukti Meta dan Medsosnya Tak Netral di Konfl...,meta perusahaan yangxamemilikixafacebook insta...,CNN Indonesia,Teknologi
3,Cara 'Kejam' Asteroid Bikin Dinosaurus Punah P...,peneliti observatorium kerajaan belgia caraxad...,CNN Indonesia,Teknologi
4,Sudah Terbukti! 2 Kegiatan Ini Bikin Anak Pint...,jakarta cnbc indonesia otak anakanak mudah men...,CNBC Indonesia,Lifestyle
5,"Semangka Jadi Simbol Dukungan ke Palestina, In...",jakarta cnbc indonesia emoji semangka bertebar...,CNBC Indonesia,Lifestyle
6,Tegas! Perusahaan RI Ini Boikot Produk Penduku...,jakarta cnbc indonesia langkah dikerahkan mend...,CNBC Indonesia,Lifestyle
7,"PDI-P Sebut Prabowo-Gibran Neo Orde Baru, Geri...",pdip prabowogibran neo orde gerindra senyumin ...,Kompas Indonesia,Nasional
8,Gerindra: Tidak Ada Istilah Prabowo Kalah di J...,gerindra istilah prabowo kalah jakarta menang ...,Kompas Indonesia,Nasional
9,Erick Thohir Diduga Merugi Kerja Politiknya Ga...,erick thohir diduga merugi politiknya gagal pi...,Kompas Indonesia,Nasional


In [ ]:
data_cleaned = data[data['Text'].str.strip() != '']
data_cleaned

,Judul,Text,Media,Label
0,Pakar Ungkap Peradaban Manusia yang Paling Ber...,peradaban mesir china mesopotamia peradaban te...,CNN Indonesia,Teknologi
1,"Daftar Kuota Internet Medsos Harian Murah, Cek...",telkomsel tri axis tercatat paket internet med...,CNN Indonesia,Teknologi
2,4 Bukti Meta dan Medsosnya Tak Netral di Konfl...,meta perusahaan yangxamemilikixafacebook insta...,CNN Indonesia,Teknologi
3,Cara 'Kejam' Asteroid Bikin Dinosaurus Punah P...,peneliti observatorium kerajaan belgia caraxad...,CNN Indonesia,Teknologi
4,Sudah Terbukti! 2 Kegiatan Ini Bikin Anak Pint...,jakarta cnbc indonesia otak anakanak mudah men...,CNBC Indonesia,Lifestyle
5,"Semangka Jadi Simbol Dukungan ke Palestina, In...",jakarta cnbc indonesia emoji semangka bertebar...,CNBC Indonesia,Lifestyle
6,Tegas! Perusahaan RI Ini Boikot Produk Penduku...,jakarta cnbc indonesia langkah dikerahkan mend...,CNBC Indonesia,Lifestyle
7,"PDI-P Sebut Prabowo-Gibran Neo Orde Baru, Geri...",pdip prabowogibran neo orde gerindra senyumin ...,Kompas Indonesia,Nasional
8,Gerindra: Tidak Ada Istilah Prabowo Kalah di J...,gerindra istilah prabowo kalah jakarta menang ...,Kompas Indonesia,Nasional
9,Erick Thohir Diduga Merugi Kerja Politiknya Ga...,erick thohir diduga merugi politiknya gagal pi...,Kompas Indonesia,Nasional


In [ ]:
# Create a new dataframe that only contain required columns
workingDF = data_cleaned[['Text', 'Media', 'Label']]
workingDF

,Text,Media,Label
0,peradaban mesir china mesopotamia peradaban te...,CNN Indonesia,Teknologi
1,telkomsel tri axis tercatat paket internet med...,CNN Indonesia,Teknologi
2,meta perusahaan yangxamemilikixafacebook insta...,CNN Indonesia,Teknologi
3,peneliti observatorium kerajaan belgia caraxad...,CNN Indonesia,Teknologi
4,jakarta cnbc indonesia otak anakanak mudah men...,CNBC Indonesia,Lifestyle
5,jakarta cnbc indonesia emoji semangka bertebar...,CNBC Indonesia,Lifestyle
6,jakarta cnbc indonesia langkah dikerahkan mend...,CNBC Indonesia,Lifestyle
7,pdip prabowogibran neo orde gerindra senyumin ...,Kompas Indonesia,Nasional
8,gerindra istilah prabowo kalah jakarta menang ...,Kompas Indonesia,Nasional
9,erick thohir diduga merugi politiknya gagal pi...,Kompas Indonesia,Nasional


In [ ]:
# Dropping index 11 and 12
workingDF = workingDF.drop([11, 12])
workingDF

,Text,Media,Label
0,peradaban mesir china mesopotamia peradaban te...,CNN Indonesia,Teknologi
1,telkomsel tri axis tercatat paket internet med...,CNN Indonesia,Teknologi
2,meta perusahaan yangxamemilikixafacebook insta...,CNN Indonesia,Teknologi
3,peneliti observatorium kerajaan belgia caraxad...,CNN Indonesia,Teknologi
4,jakarta cnbc indonesia otak anakanak mudah men...,CNBC Indonesia,Lifestyle
5,jakarta cnbc indonesia emoji semangka bertebar...,CNBC Indonesia,Lifestyle
6,jakarta cnbc indonesia langkah dikerahkan mend...,CNBC Indonesia,Lifestyle
7,pdip prabowogibran neo orde gerindra senyumin ...,Kompas Indonesia,Nasional
8,gerindra istilah prabowo kalah jakarta menang ...,Kompas Indonesia,Nasional
9,erick thohir diduga merugi politiknya gagal pi...,Kompas Indonesia,Nasional


In [ ]:
# Rearrange the index
workingDF = workingDF.reset_index(drop=True)
workingDF

,Text,Media,Label
0,peradaban mesir china mesopotamia peradaban te...,CNN Indonesia,Teknologi
1,telkomsel tri axis tercatat paket internet med...,CNN Indonesia,Teknologi
2,meta perusahaan yangxamemilikixafacebook insta...,CNN Indonesia,Teknologi
3,peneliti observatorium kerajaan belgia caraxad...,CNN Indonesia,Teknologi
4,jakarta cnbc indonesia otak anakanak mudah men...,CNBC Indonesia,Lifestyle
5,jakarta cnbc indonesia emoji semangka bertebar...,CNBC Indonesia,Lifestyle
6,jakarta cnbc indonesia langkah dikerahkan mend...,CNBC Indonesia,Lifestyle
7,pdip prabowogibran neo orde gerindra senyumin ...,Kompas Indonesia,Nasional
8,gerindra istilah prabowo kalah jakarta menang ...,Kompas Indonesia,Nasional
9,erick thohir diduga merugi politiknya gagal pi...,Kompas Indonesia,Nasional


## Vectorization

In [ ]:
# TF-IDF
tfidf_vectorizer = TfidfVectorizer(min_df=3, max_features=50)
tfidf_vectors = tfidf_vectorizer.fit_transform(workingDF['Text'])
workingDF['TF-IDF'] = list(tfidf_vectors.toarray())

# Word2Vec
sentences = [text.split() for text in workingDF['Text']]
model = Word2Vec(sentences, vector_size=50, min_count=3)

def get_vector(text):
    words = text.split()
    vec = np.zeros(50)
    count = 0
    for word in words:
        if word in model.wv:
            vec += model.wv[word]
            count += 1
    if count != 0:
        vec /= count
    return vec

workingDF['Word2Vec'] = workingDF['Text'].apply(get_vector)
workingDF

,Text,Media,Label,TF-IDF,Word2Vec
0,peradaban mesir china mesopotamia peradaban te...,CNN Indonesia,Teknologi,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0051385448314249516, 0.01623336598277092, -..."
1,telkomsel tri axis tercatat paket internet med...,CNN Indonesia,Teknologi,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,meta perusahaan yangxamemilikixafacebook insta...,CNN Indonesia,Teknologi,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0]","[-0.014278030022978783, 0.0024820612743496895,..."
3,peneliti observatorium kerajaan belgia caraxad...,CNN Indonesia,Teknologi,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,jakarta cnbc indonesia otak anakanak mudah men...,CNBC Indonesia,Lifestyle,"[0.0, 0.6448594488714666, 0.540442546068122, 0...","[0.007954808839713223, -0.0013895596687992413,..."
5,jakarta cnbc indonesia emoji semangka bertebar...,CNBC Indonesia,Lifestyle,"[0.0, 0.5419477406385818, 0.45419450284733365,...","[0.002396599124040222, -0.00042165443301200867..."
6,jakarta cnbc indonesia langkah dikerahkan mend...,CNBC Indonesia,Lifestyle,"[0.0, 0.5419477406385818, 0.45419450284733365,...","[0.002396599124040222, -0.00042165443301200867..."
7,pdip prabowogibran neo orde gerindra senyumin ...,Kompas Indonesia,Nasional,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
8,gerindra istilah prabowo kalah jakarta menang ...,Kompas Indonesia,Nasional,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0]","[0.01563364453613758, -0.019020093604922295, -..."
9,erick thohir diduga merugi politiknya gagal pi...,Kompas Indonesia,Nasional,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


## Modelling

In [ ]:
# Splotting dataset into 80% training and 20% test
X_word2vec = np.vstack(workingDF['Word2Vec'].to_numpy())
X_tfidf = np.vstack(workingDF['TF-IDF'].to_numpy())
y = workingDF['Label']

X_train_word2vec, X_test_word2vec, y_train, y_test = train_test_split(X_word2vec, y, test_size=0.2, random_state=42)
X_train_tfidf, X_test_tfidf, _, _ = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

In [ ]:
# undersampling using Ranom undersampler
rus = RandomUnderSampler(random_state=42)
X_train_word2vec_resampled, y_train_resampled = rus.fit_resample(X_train_word2vec, y_train)
X_train_tfidf_resampled, y_train_resampled = rus.fit_resample(X_train_tfidf, y_train)

In [ ]:
# Train the SVM model with Word2Vec
svm_word2vec = SVC(kernel='linear', C=1)
svm_word2vec.fit(X_train_word2vec_resampled, y_train_resampled)

# Train the SVM model with TF-IDF
svm_tfidf = SVC(kernel='linear', C=1)
svm_tfidf.fit(X_train_tfidf_resampled, y_train_resampled)

# Predict on the test set
y_pred_word2vec = svm_word2vec.predict(X_test_word2vec)
y_pred_tfidf = svm_tfidf.predict(X_test_tfidf)

In [ ]:
# Calculating Accuracy
accuracy_word2vec = accuracy_score(y_test, y_pred_word2vec)
accuracy_tfidf = accuracy_score(y_test, y_pred_tfidf)

# Calculating Precision and Recall
precision_word2vec = precision_score(y_test, y_pred_word2vec, average='macro', zero_division=1)
recall_word2vec = recall_score(y_test, y_pred_word2vec, average='macro', zero_division=1)

precision_tfidf = precision_score(y_test, y_pred_tfidf, average='macro', zero_division=1)
recall_tfidf = recall_score(y_test, y_pred_tfidf, average='macro', zero_division=1)

print("SVM - Word2Vec")
print("Akurasi\t\t:", accuracy_word2vec* 100, "%")
print("Precision\t:", precision_word2vec* 100, "%")
print("Recall\t\t:", recall_word2vec * 100, "%")
print("\n")
print("SVM - TF-IDF")
print("Akurasi\t\t:", accuracy_tfidf * 100, "%")
print("Precision\t:", precision_tfidf * 100, "%")
print("Recall\t\t:", recall_tfidf * 100, "%")

SVM - Word2Vec
Akurasi		: 75.0 %
Precision	: 88.88888888888889 %
Recall		: 66.66666666666666 %


SVM - TF-IDF
Akurasi		: 50.0 %
Precision	: 75.0 %
Recall		: 75.0 %


In [ ]:
#Train the RandomForest model with Word2Vec
rf_word2vec = RandomForestClassifier(n_estimators=100, random_state=42)
rf_word2vec.fit(X_train_word2vec_resampled, y_train_resampled)

#Train the RandomForest model with TF-IDF
rf_tfidf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_tfidf.fit(X_train_tfidf_resampled, y_train_resampled)# Predict with Word2Vec

# Predict on the test set
y_pred_rf_word2vec = rf_word2vec.predict(X_test_word2vec)
y_pred_rf_tfidf = rf_tfidf.predict(X_test_tfidf)

In [ ]:
# Calculating the Accureacy
accuracy_rf_word2vec = accuracy_score(y_test, y_pred_rf_word2vec)
accuracy_rf_tfidf = accuracy_score(y_test, y_pred_rf_tfidf)

# Calculating Precision and Recall
precision_rf_word2vec = precision_score(y_test, y_pred_rf_word2vec, average='macro', zero_division=1)
recall_rf_word2vec = recall_score(y_test, y_pred_rf_word2vec, average='macro', zero_division=1)

precision_rf_tfidf = precision_score(y_test, y_pred_rf_tfidf, average='macro', zero_division=1)
recall_rf_tfidf = recall_score(y_test, y_pred_rf_tfidf, average='macro', zero_division=1)

print("Random Forest - Word2Vec")
print("Akurasi\t\t:", accuracy_rf_word2vec * 100, "%")
print("Precision\t:", precision_rf_word2vec * 100, "%")
print("Recall\t\t:", recall_rf_word2vec * 100, "%")
print("\n")
print("Random Forest - TF-IDF")
print("Akurasi\t\t:", accuracy_rf_tfidf * 100, "%")
print("Precision\t:", precision_rf_tfidf * 100, "%")
print("Recall\t\t:", recall_rf_tfidf * 100, "%")

Random Forest - Word2Vec
Akurasi		: 50.0 %
Precision	: 50.0 %
Recall		: 50.0 %


Random Forest - TF-IDF
Akurasi		: 100.0 %
Precision	: 100.0 %
Recall		: 100.0 %


## Conclusion